In [ ]:
import torch
import torchvision
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import argparse
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import random

from Extract_mouse_data import Mouse_sub_volumes
from data_augmentation import Rotate, Flip
from weight_init import weight_init
from my_net import VGG_net

In [ ]:
#####Function and class here
def get_subbox_idx(input_idx,input_all_data):
    sub_box=[]
    neg_num=0
    pos_num=0
    for idx in input_idx:
        for i in range(len(input_all_data[idx][0])):
            sub_box.append((idx,0,input_all_data[idx][0][i]))
        neg_num+=len(input_all_data[idx][0])
        for i in range(len(input_all_data[idx][1])):
            sub_box.append((idx,1,input_all_data[idx][1][i]))
        pos_num+=len(input_all_data[idx][1])
    print(neg_num)
    print(pos_num)
    print(len(sub_box))
    return sub_box

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    for i_batch, sample_batched in enumerate(train_loader):
        inputs, labels = sample_batched['image'], sample_batched['label']  
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i_batch % 10 == 0:
            print("epoch {}, batch {}, current loss {}".format(epoch+1,i_batch,running_loss/10))
            running_loss = 0.0

def test(model, device, test_loader):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    
    
def test_ensemble(model_list, device, test_loader):
    for i in range(len(model_list)):
        model_list[i].eval()
    #model3.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = torch.nn.functional.softmax(model_list[0](inputs),dim=1)
            for i in range(1,len(model_list)):
                outputs += torch.nn.functional.softmax(model_list[i](inputs),dim=1)
            _, predicted = torch.max(outputs, 1)
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))




def load_data(path):
    with open(data_path,'rb') as f:
        #(neg_subvolumes,pos_subvolumes,img2,filtered_img2,img_label2,data_dic[i][0])
        all_data = pickle.load(f)
    f.close()
    return all_data

In [ ]:
def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch BV classifier.')
    parser.add_argument('--batch_size', type=int, default=100, metavar='N',
                        help='input batch size for training (default: 100)')
    parser.add_argument('--epochs', type=int, default=7, metavar='N',
                        help='number of epochs to train (default: 7)')
    parser.add_argument('--lr', type=float, default=0.005, metavar='LR',
                        help='learning rate (default: 0.005)')
    parser.add_argument('--SGD', action='store_true',
                       help='set optimizer as SGD')
    parser.add_argument('--Adam', action='store_true'
                       help='set optimizer as Adam')
    parser.add_argument('--RMSprop', action='store_true'
                       help='set optimizer as RMSprop')
    parser.add_argument('--save_version', type=int, default=1,
                        help='save_version (default 1)')
    args = parser.parse_args()
    print("batch_size: {}, training epochs: {}, learning rate: {}, SGD {}, Adam {}, RMSprop {}, save_version {}".
         format(args.batch_szie, args.epochs, args.lr, args.SGD, args.Adam, args.RMSprop, args.save_version))
    
    data_path = os.path.join(os.getcwd(),'data','2018_0622_all_sub_volumes.pickle')
    all_data = load_data(data_path)
    
    HALF_SIDE = 64 # half of the bounding size 128
    #take 1/6 as test img and the rest as train img
    test_idx = list(range(0,len(all_data),6))
    total_idx = list(range(len(all_data)))
    train_idx = [x for x in total_idx if x not in test_idx]

    print("test index: {0}".format(test_idx))
    print("train index: {0}".format(train_idx))
    
    all_whole_volumes = {}
    all_whole_filtered_volumes = {}
    for i in range(len(all_data)):
        all_whole_volumes[i] = all_data[i][2]
        all_whole_filtered_volumes[i] = all_data[i][3]
        
    train_sub_box=get_subbox_idx(train_idx,all_data)
    test_sub_box=get_subbox_idx(test_idx,all_data)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net = VGG_net()
    #net.apply(weight_init)
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        net = nn.DataParallel(net)
    net.to(device)
    print("There are {} parameters in the model".format(count_parameters(net)))
    
    criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([1,1.2]).to(device))
    if args.SGD == True:
        optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=0.00001)
        print('choose SGD as optimizer')
    elif args.Adam == True:
        optimizer = optim.Adam(net.parameters(), lr=args.lr*10, weight_decay=0.00001)
        print('choose Adam as optimizer')
    elif args.RMSprop == True:
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr*10, weight_decay=0.00001)
        print('choose RMSprop as optimizer')
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    
    for epoch in range(args.epochs):
        scheduler.step()
        Mouse_dataset = Mouse_sub_volumes(all_whole_volumes,all_whole_filtered_volumes,train_sub_box,
                                     transform=transforms.Compose([Rotate(),Flip()]))
        dataloader = DataLoader(Mouse_dataset, batch_size=args.batch_size,
                        shuffle=True, num_workers=4)
        train(net, device, dataloader, optimizer, criterion, epoch)
    print('Finished Training')
    torch.save(net.state_dict(), './model/bv_classifier_b{}_e{}_v{}.pth'.format(args.batch_size,
                                                                             args.epochs,
                                                                             args.save_version))
    #####test
    print('train accuracy: ')
    Mouse_dataset = Mouse_sub_volumes(all_whole_volumes,all_whole_filtered_volumes,train_sub_box)
    train_dataloader = DataLoader(Mouse_dataset, batch_size=args.batch_size,
                        shuffle=False, num_workers=4)
    test(net, device, train_dataloader)
    
    print('test accuracy: ')
    Mouse_dataset = Mouse_sub_volumes(all_whole_volumes,all_whole_filtered_volumes,test_sub_box)
    test_dataloader = DataLoader(Mouse_dataset, batch_size=args.batch_size,
                        shuffle=False, num_workers=4)
    test(net, device, test_dataloader)
if __name__ == '__main__':
    main()

In [ ]:
data_path = os.path.join(os.getcwd(),'data','2018_0622_all_sub_volumes.pickle')
all_data = load_data(data_path)
    
HALF_SIDE = 64 # half of the bounding size 128
#take 1/6 as test img and the rest as train img
test_idx = list(range(0,len(all_data),6))
total_idx = list(range(len(all_data)))
train_idx = [x for x in total_idx if x not in test_idx]

print("test index: {0}".format(test_idx))
print("train index: {0}".format(train_idx))
    
all_whole_volumes = {}
all_whole_filtered_volumes = {}
for i in range(len(all_data)):
    all_whole_volumes[i] = all_data[i][2]
    all_whole_filtered_volumes[i] = all_data[i][3]
        
train_sub_box=get_subbox_idx(train_idx,all_data)
test_sub_box=get_subbox_idx(test_idx,all_data)
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net1 = VGG_net()
net2 = VGG_net()
net3 = VGG_net()
net4 = VGG_net()
net5 = VGG_net()
net6 = VGG_net()
net7 = VGG_net()
#net.apply(weight_init)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net1 = nn.DataParallel(net1)
    net2 = nn.DataParallel(net2)
    net3 = nn.DataParallel(net3)
    net4 = nn.DataParallel(net4)
    net5 = nn.DataParallel(net5)
    net6 = nn.DataParallel(net6)
    net7 = nn.DataParallel(net7)
    
net1.to(device)
net1.load_state_dict(torch.load('./model/bv_classifier_b200_e5_v8.pth'))
net2.to(device)
net2.load_state_dict(torch.load('./model/bv_classifier_b200_e5_v9.pth'))
net3.to(device)
net3.load_state_dict(torch.load('./model/bv_classifier_b200_e7_v3.pth'))
net4.to(device)
net4.load_state_dict(torch.load('./model/bv_classifier_b200_e7_v4.pth'))
net5.to(device)
net5.load_state_dict(torch.load('./model/bv_classifier_b200_e7_v5.pth'))
net6.to(device)
net6.load_state_dict(torch.load('./model/bv_classifier_b200_e7_v6.pth'))
net7.to(device)
net7.load_state_dict(torch.load('./model/bv_classifier_b200_e7_v7.pth'))
net = [net1,net2,net5,net6,net7]
print("There are {} parameters in the model".format(count_parameters(net1)))

#####test
print('test: ')
Mouse_dataset = Mouse_sub_volumes(all_whole_volumes,all_whole_filtered_volumes,test_sub_box)
test_dataloader = DataLoader(Mouse_dataset, batch_size=200,
                        shuffle=False, num_workers=4)
#test(net1, device, test_dataloader)
#test(net2, device, test_dataloader)
test_ensemble(net, device, test_dataloader)

In [ ]:
import dense_net
data_path = os.path.join(os.getcwd(),'data','2018_0622_all_sub_volumes.pickle')
all_data = load_data(data_path)
    
HALF_SIDE = 64 # half of the bounding size 128
#take 1/6 as test img and the rest as train img
test_idx = list(range(0,len(all_data),6))
total_idx = list(range(len(all_data)))
train_idx = [x for x in total_idx if x not in test_idx]

print("test index: {0}".format(test_idx))
print("train index: {0}".format(train_idx))
    
all_whole_volumes = {}
all_whole_filtered_volumes = {}
for i in range(len(all_data)):
    all_whole_volumes[i] = all_data[i][2]
    all_whole_filtered_volumes[i] = all_data[i][3]
        
train_sub_box=get_subbox_idx(train_idx,all_data)
test_sub_box=get_subbox_idx(test_idx,all_data)
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = dense_net.DenseNet()
#net.apply(weight_init)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
net.to(device)
print("There are {} parameters in the model".format(count_parameters(net)))
    
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([1,1.2]).to(device))
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    
for epoch in range(5):
    scheduler.step()
    Mouse_dataset = Mouse_sub_volumes(all_whole_volumes,all_whole_filtered_volumes,train_sub_box,
                                     transform=transforms.Compose([Rotate(),Flip()]))
    dataloader = DataLoader(Mouse_dataset, batch_size=100,
                        shuffle=True, num_workers=4)
    train(net, device, dataloader, optimizer, criterion, epoch)

In [ ]:
Mouse_dataset = Mouse_sub_volumes(all_whole_volumes,all_whole_filtered_volumes,test_sub_box)
test_dataloader = DataLoader(Mouse_dataset, batch_size=200,
                        shuffle=False, num_workers=4)
test(net, device, test_dataloader)

In [ ]:
Mouse_dataset = Mouse_sub_volumes(all_whole_volumes,all_whole_filtered_volumes,train_sub_box)
test_dataloader = DataLoader(Mouse_dataset, batch_size=200,
                        shuffle=False, num_workers=4)
test(net, device, test_dataloader)

In [1]:

# coding: utf-8

# In[1]:


import torch
import torchvision
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import argparse
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import random

from Extract_mouse_data import Mouse_sub_volumes
from data_augmentation import Rotate, Flip
from weight_init import weight_init
from my_net import VGG_net


# In[2]:


#####Function and class here
def get_subbox_idx(input_idx,input_all_data):
    sub_box=[]
    neg_num=0
    pos_num=0
    for idx in input_idx:
        for i in range(len(input_all_data[idx][0])):
            sub_box.append((idx,0,input_all_data[idx][0][i]))
        neg_num+=len(input_all_data[idx][0])
        for i in range(len(input_all_data[idx][1])):
            sub_box.append((idx,1,input_all_data[idx][1][i]))
        pos_num+=len(input_all_data[idx][1])
    print(neg_num)
    print(pos_num)
    print(len(sub_box))
    return sub_box

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    for i_batch, sample_batched in enumerate(train_loader):
        inputs, labels = sample_batched['image'], sample_batched['label']  
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i_batch % 1 == 0:
            print("epoch {}, batch {}, current loss {}".format(epoch+1,i_batch,running_loss/1))
            running_loss = 0.0

def test(model, device, test_loader):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    
def test_ensemble(model_list, device, test_loader):
    for model in model_list:
        model.eval()
    #model3.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = model_list[0](inputs)
            for i in range(1,len(model_list)):
                outputs += model_list[i](inputs)
            _, predicted = torch.max(outputs, 1)
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))



def load_data(data_path):
    with open(data_path,'rb') as f:
        #(neg_subvolumes,pos_subvolumes,img2,filtered_img2,img_label2,data_dic[i][0])
        all_data = pickle.load(f)
    f.close()
    return all_data


# In[ ]:

class Argument():
    def __init__(self):
        self.batch_size = 200
        self.epochs = 5
        self.lr = 0.01
        self.SGD = True
        self.Adam = False
        self.RMSprop = False
        self.save_version = 1
        
def main():
    # Training settings    
    args = Argument()
    print("batch_size: {}, training epochs: {}, learning rate: {}, SGD {}, Adam {}, RMSprop {}, save_version {}".
         format(args.batch_size, args.epochs, args.lr, args.SGD, args.Adam, args.RMSprop, args.save_version))
    
    data_path = os.path.join(os.getcwd(),'data','2018_0711_train_sub_volumes2.pickle')
    #(neg_subvolumes,pos_subvolumes,img2,filtered_img2, img_label2, 
    # img, filtered_img, img_label, data_dic[i][0])
    all_data = load_data(data_path)
    
    HALF_SIDE = 64 # half of the bounding size 128
    #take 1/6 as test img and the rest as train img
    test_idx = list(range(0,len(all_data),6))
    total_idx = list(range(len(all_data)))
#    train_idx = [x for x in total_idx if x not in test_idx]

    print("test index: {0}".format(test_idx))
    print("train index: {0}".format(total_idx))
    
    all_whole_volumes = {}
    all_whole_filtered_volumes = {}
    for i in range(len(all_data)):
        all_whole_volumes[i] = all_data[i][2] - 0.5
        all_whole_filtered_volumes[i] = all_data[i][3] - 0.5
        
    train_sub_box=get_subbox_idx(total_idx,all_data)
    test_sub_box=get_subbox_idx(test_idx,all_data)

    del all_data
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net = VGG_net()
    #net.apply(weight_init)
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        net = nn.DataParallel(net)
    net.to(device)
    print("There are {} parameters in the model".format(count_parameters(net)))
    
    criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([1,1.2]).to(device))
    if args.SGD == True:
        optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=0.00001)
        print('choose SGD as optimizer')
    elif args.Adam == True:
        optimizer = optim.Adam(net.parameters(), lr=args.lr*10, weight_decay=0.00001)
        print('choose Adam as optimizer')
    elif args.RMSprop == True:
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr*10, weight_decay=0.00001)
        print('choose RMSprop as optimizer')
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    
    for epoch in range(args.epochs):
        scheduler.step()
        Mouse_dataset = Mouse_sub_volumes(all_whole_volumes,all_whole_filtered_volumes,train_sub_box,
                                     transform=transforms.Compose([Rotate(),Flip()]))
        dataloader = DataLoader(Mouse_dataset, batch_size=args.batch_size,
                        shuffle=True, num_workers=4)
        train(net, device, dataloader, optimizer, criterion, epoch)
    print('Finished Training')
    torch.save(net.state_dict(), './model/all_bv_classifier64_c2_b{}_e{}_v{}.pth'.format(
                                                                             args.batch_size,
                                                                             args.epochs,
                                                                             args.save_version))
    #####test
    print('train accuracy: ')
    Mouse_dataset = Mouse_sub_volumes(all_whole_volumes,all_whole_filtered_volumes,train_sub_box)
    train_dataloader = DataLoader(Mouse_dataset, batch_size=args.batch_size,
                        shuffle=False, num_workers=4)
    test(net, device, train_dataloader)
    
    print('test accuracy: ')
    Mouse_dataset = Mouse_sub_volumes(all_whole_volumes,all_whole_filtered_volumes,test_sub_box)
    test_dataloader = DataLoader(Mouse_dataset, batch_size=args.batch_size,
                        shuffle=False, num_workers=4)
    test(net, device, test_dataloader)
if __name__ == '__main__':
    main()

batch_size: 200, training epochs: 5, learning rate: 0.01, SGD True, Adam False, RMSprop False, save_version 1
test index: [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240, 246, 252, 258]
train index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 15

epoch 1, batch 129, current loss 0.47304022312164307
epoch 1, batch 130, current loss 0.44413235783576965
epoch 1, batch 131, current loss 0.42532768845558167
epoch 1, batch 132, current loss 0.4883418083190918
epoch 1, batch 133, current loss 0.3542795479297638
epoch 1, batch 134, current loss 0.3595634400844574
epoch 1, batch 135, current loss 0.441104918718338
epoch 1, batch 136, current loss 0.48399782180786133
epoch 1, batch 137, current loss 0.45845791697502136
epoch 1, batch 138, current loss 0.44914183020591736
epoch 1, batch 139, current loss 0.4222983717918396
epoch 1, batch 140, current loss 0.4989639222621918
epoch 1, batch 141, current loss 0.37319809198379517
epoch 1, batch 142, current loss 0.3679457902908325
epoch 1, batch 143, current loss 0.4275304973125458
epoch 1, batch 144, current loss 0.45655056834220886
epoch 1, batch 145, current loss 0.4302317798137665
epoch 1, batch 146, current loss 0.4001675546169281
epoch 1, batch 147, current loss 0.37193435430526733
epoc

Process Process-1:
Process Process-4:
Process Process-3:
Process Process-2:
Traceback (most recent call last):
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/scratch/zq415/grammar_cor/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/scratch/zq415/grammar_cor/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/scratch/zq415/grammar_cor/Localization/Extract_mouse_data.py", line 36, in __getitem__
    z-box_size:z]
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recent call last):
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/mu

KeyboardInterrupt: 